In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
sess = tf.Session()
optim = tf.train.AdamOptimizer(learning_rate=0.001)

In [4]:
from tqdm import tqdm

In [5]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [6]:
import BayesianNN

In [7]:
bnn = BayesianNN.BayesianFC(1,1)

In [8]:
scale = 1

In [9]:
data = [(x,scale*x) for x in range(0,1000)]

In [10]:
data_norm = [([a/1000],[b/1000]) for (a,b) in data]
data_input = list(map(lambda x: x[0], data_norm))
data_target = list(map(lambda x: x[1], data_norm))

In [11]:
sess.run(tf.initialize_all_variables())
sess.run(tf.initialize_variables(optim.get_slot_names()))

In [18]:
# print(sess.run(bnn.qw_mean))
batch_size = 100
op, x, y, grads = bnn.update(3, optim)
for e in tqdm(range(10000)):
#     indices = np.random.randint(low=0, high=100, size=batch_size)
#     input_batch = [data_input[i] for i in indices]
#     target_batch = [data_target[i] for i in indices]
    input_batch = data_input
    target_batch = data_target
    
    grad_nums = [a for (a,b) in grads]
    _, g = sess.run([op, grad_nums], feed_dict={x: input_batch, y:target_batch})
#     print()
#     print(g)
#     print()
        

#     print("Epoch {}:".format(e))
#     print(sess.run([bnn.qb_mean, bnn.qb_p]))
#     print(g)
#     print()
    #     print("---")
    #     print(sess.run([bnn.qw_p, bnn.qb_p]))    

100%|██████████| 10000/10000 [00:16<00:00, 609.57it/s]


In [19]:
test_set = [[x/100] for x in range(-100, 200)]
test_set_flat = [a[0] for a in test_set]

In [20]:
predictions = sess.run(bnn.output(test_set, sample=True))
var_trials = 100
for _ in range(var_trials-1):
    predictions = np.concatenate([predictions, sess.run(bnn.output(test_set, sample=True))], axis=1)

In [21]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)

In [22]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [23]:
p = figure(width=500, height=500)
err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.3))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")
p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")

show(p)